<a href="https://colab.research.google.com/github/eisbetterthanpi/RNN/blob/main/RNNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q torchdata portalocker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 42.0 MB/s eta 0:00:00


In [ ]:
# @title data
import requests
# url="https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt" # https://www.tensorflow.org/text/tutorials/text_generation
url="https://raw.githubusercontent.com/wojzaremba/lstm/master/data/ptb.train.txt" # train test valid # https://pytorch.org/text/stable/datasets.html#penntreebank
out=requests.get(url)
with open("data.txt", "wb") as f:
    f.write(out.content)
text = open("data.txt", 'rb').read().decode(encoding='utf-8')

url="https://raw.githubusercontent.com/wojzaremba/lstm/master/data/ptb.test.txt" # train test valid # https://pytorch.org/text/stable/datasets.html#penntreebank
out=requests.get(url)
with open("data_.txt", "wb") as f: f.write(out.content)
test_text = open("data_.txt", 'rb').read().decode(encoding='utf-8')

# print(len(text))
# print(text[000:1000])
# data = ''.join(text)
# chars = sorted(list(set(data)))
# print(chars)


In [ ]:
# @title dataloader
# https://github.com/Sam-Armstrong/tinyGPT/blob/main/Training.py
# https://colab.research.google.com/github/karpathy/minGPT/blob/master/play_char.ipynb
# https://github.com/karpathy/nanoGPT
import torch
import torch.nn as nn
from torch.utils.data import Dataset

class CharDataset(Dataset): # https://github.com/karpathy/minGPT
    def __init__(self, raw_data, seq_len):
        data = ''.join(raw_data)
        chars = sorted(list(set(data)))
        self.vocab_size = len(chars) # 283
        self.stoi = {ch:i for i,ch in enumerate(chars)}
        self.itos = {i:ch for i,ch in enumerate(chars)}
        self.data = self.data_process(data) # list of int
        self.seq_len = seq_len

    def data_process(self, data): # str 10780437
        return torch.tensor([self.stoi.get(c) for c in data]) # list of int 4570571 # stoi.get(c,UNK_IDX)

    def __len__(self):
        # return len(self.data) - self.seq_len
        return len(self.data)//(self.seq_len+1)

    def __getitem__(self, idx):
        # dix = self.data[idx:idx + self.seq_len + 1]
        dix = self.data[idx*(self.seq_len+1) : (idx+1)*(self.seq_len+1)]
        x, y = dix[:-1], dix[1:]
        return x, y


seq_len = 100 # 128
train_dataset = CharDataset(text, seq_len) # one line of poem is roughly 50 characters
# test_dataset = CharDataset(test_text, seq_len) # one line of poem is roughly 50 characters
from torch.utils.data.dataloader import DataLoader
batch_size = 64 #512
train_loader = DataLoader(train_dataset, shuffle = True, pin_memory = True, batch_size = batch_size, num_workers = 2) # num_workers = 4
# test_loader = DataLoader(test_dataset, shuffle = True, pin_memory = True, batch_size = batch_size, num_workers = 0)


def encode(context): return torch.tensor([train_dataset.stoi.get(c) for c in context], device=device)
def decode(x): return ''.join([train_dataset.itos[int(i)] for i in x])
# for x,y in train_loader:
#     break


In [ ]:
# @title RNN from scratch
# https://github.com/georgeyiasemis/Recurrent-Neural-Networks-from-scratch-using-PyTorch/blob/main/rnncells.py
import torch
import torch.nn as nn
device = "cuda" if torch.cuda.is_available() else "cpu"

class RNNCell(nn.Module):
    def __init__(self, in_dim, d_model, bias=True):
        super().__init__()
        self.d_model = d_model
        self.x2h = nn.Linear(in_dim, d_model, bias=bias)
        self.h2h = nn.Linear(d_model, d_model, bias=bias)

    def forward(self, input, hx=None):
        if hx is None: hx = torch.zeros(input.size(0), self.d_model, device=device)
        hy = self.x2h(input) + self.h2h(hx)
        hy = torch.tanh(hy)
        return hy

class LSTMCell(nn.Module):
    def __init__(self, in_dim, d_model, bias=True):
        super().__init__()
        self.d_model = d_model
        self.xh = nn.Linear(in_dim, 4*d_model, bias=bias)
        self.hh = nn.Linear(d_model, 4*d_model, bias=bias)

    def forward(self, input, hxcx=None):
        if hxcx is None: hx, cx = torch.zeros(input.size(0), self.d_model, device=device), torch.zeros(input.size(0), self.d_model, device=device)
        else: hx, cx = hxcx
        gates = self.xh(input) + self.hh(hx) # gates = self.xh(torch.cat([input, hx], dim=-1))
        input_gate, forget_gate, cell_gate, output_gate = gates.chunk(4, dim=-1)
        f_t = torch.sigmoid(forget_gate)
        i_t = torch.sigmoid(input_gate)
        g_t = torch.tanh(cell_gate)
        o_t = torch.sigmoid(output_gate)
        cy = cx * f_t + i_t * g_t
        hy = o_t * torch.tanh(cy)
        return hy, cy

class GRUCell(nn.Module):
    def __init__(self, in_dim, d_model, bias=True):
        super().__init__()
        self.d_model = d_model
        self.x2h = nn.Linear(in_dim, 3*d_model, bias=bias)
        self.h2h = nn.Linear(d_model, 3*d_model, bias=bias)

    def forward(self, input, hx=None): # input: (batch_size, in_dim), hx: (batch_size, d_model)
        if hx is None: hx = torch.zeros(input.size(0), self.d_model, device=device)
        x_t = self.x2h(input)
        h_t = self.h2h(hx)
        x_reset, x_upd, x_new = x_t.chunk(3, dim=-1)
        h_reset, h_upd, h_new = h_t.chunk(3, dim=-1)
        reset_gate = torch.sigmoid(x_reset + h_reset)
        update_gate = torch.sigmoid(x_upd + h_upd)
        new_gate = torch.tanh(x_new + reset_gate*h_new)
        hy = update_gate*hx + (1-update_gate)*new_gate
        return hy # (batch_size, d_model)


class RNN(nn.Module): # https://github.com/georgeyiasemis/Recurrent-Neural-Networks-from-scratch-using-PyTorch/blob/main/rnnmodels.py
    def __init__(self, in_dim, d_model, num_layers, out_dim):
        super().__init__()
        self.num_layers = num_layers
        self.d_model = d_model
        self.tok_emb = nn.Embedding(in_dim, d_model)
        # self.rnn = nn.ModuleList([RNNCell(d_model, d_model) for _ in range(num_layers)])
        self.rnn = nn.ModuleList([LSTMCell(d_model, d_model) for _ in range(num_layers)])
        # self.rnn = nn.ModuleList([GRUCell(d_model, d_model) for _ in range(num_layers)])
        self.fc = nn.Linear(d_model, out_dim)

        # std = 1.0 / torch.sqrt(torch.tensor(self.d_model))
        # for w in self.parameters():
        #     w.data.uniform_(-std, std)
        #     # nn.init.uniform_(w, -std, std)
        # nn.init.xavier_uniform_(self)

        # for param in self.parameters():
        #     if param.dim() > 1:
        #         nn.init.xavier_uniform_(param)

        # def init_weights(self):
        #     for name, param in self.named_parameters():
        #         if "weight" in name:
        #             nn.init.xavier_uniform_(param)
        #         elif "bias" in name:
        #             nn.init.zeros_(param)

    def forward(self, x, h0=None):
        batch_size, seq_len = x.shape
        x = self.tok_emb(x) #[batch_size, seq_len, d_model]
        if h0==None: h0 = torch.zeros(self.num_layers, batch_size, self.d_model, device=device)
        ht = list(torch.split(h0, 1))
        out = []
        for t in range(seq_len):
            for i, layer in enumerate(self.rnn):
                if i==0: ht[i] = layer(x[:,t], h0[i])
                else: ht[i] = layer(ht[i-1], h0[i])
            out.append(ht[-1]) # seq_len * [batch_size, d_model]
            h0 = ht
        ht = torch.stack(ht, dim=0)
        out = torch.stack(out, dim=1) # [batch_size, seq_len, d_model]
        # out=h0[-1]

        # for t in range(seq_len): # simple single layer
        #     h0 = self.rnn(x[:, t], h0)
        #     out.append(h0)

        out = self.fc(out) # out: (n, 10)
        return out, ht # rnn/gru


    def forward(self, x, hc=None): # lstm
        batch_size, seq_len = x.shape
        x = self.tok_emb(x) #[batch_size, seq_len, d_model]
        if hc==None:
            h0 = torch.zeros(self.num_layers, batch_size, self.d_model, device=device)
            c0 = torch.zeros(self.num_layers, batch_size, self.d_model, device=device)
        else: h0, c0 = hc
        ht, ct = list(torch.split(h0, 1)), list(torch.split(c0, 1))
        out = []
        for t in range(seq_len):
            for i, layer in enumerate(self.rnn):
                if i==0: ht[i], ct[i] = layer(x[:, t], (h0[i],c0[i]))
                else: ht[i], ct[i] = layer(ht[i-1], (h0[i],c0[i]))
            out.append(ht[-1]) # seq_len * [batch_size, d_model]
            h0 = ht
        ht, ct = torch.stack(ht, dim=0), torch.stack(ct, dim=0)
        out = torch.stack(out, dim=1) # [batch_size, seq_len, d_model]
        out = self.fc(out) # out: (n, 10)
        return out, (ht, ct) # lstm


hidden_size = 128#128
num_layers = 2#2
input_size = num_classes = train_dataset.vocab_size#65

model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)



65


In [ ]:
# @title RNN pytorch
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = "cuda" if torch.cuda.is_available() else "cpu"

import math

class RNN(nn.Module):
    def __init__(self, in_dim, d_model, out_dim=None, num_layers=1):
        super().__init__()
        if out_dim is None: out_dim = in_dim
        self.num_layers = num_layers
        self.d_model = d_model
        # self.rnn = nn.RNN(d_model, d_model, num_layers, batch_first=True)
        self.rnn = nn.GRU(d_model, d_model, num_layers, batch_first=True)
        # self.lstm = nn.LSTM(d_model, d_model, num_layers, batch_first=True)
        self.fc = nn.Linear(d_model, out_dim)

        self.emb = nn.Embedding(in_dim, d_model)
        # self.d_model = d_model
        # self.fc = nn.Linear(d_model, vocab_size)
        # for p in self.parameters():
        #     if p.dim() > 1:
        #         nn.init.xavier_uniform_(p)

    def forward(self, x, hc=None): # lstm [batch_size, seq, in_dim]
        x = self.emb(x)
        if hc is None:
            h0 = torch.zeros((self.num_layers, x.size(0), self.d_model), device=device)
            c0 = torch.zeros((self.num_layers, x.size(0), self.d_model), device=device)
        else: h0,c0 = hc
        # print(x.shape, h0.shape,c0.shape)
        out, (h,c) = self.lstm(x, (h0,c0)) # [batch, seq_len, d_model], ([num_layers, batch, d_model] )
        # out = out[:, -1, :] # out: (n, 128)
        out = self.fc(out) # out: (n, 10)
        return out, (h, c)

    def forward(self, x, h=None): # rnn/gru
        x = self.emb(x)# * math.sqrt(self.d_model)
        if h is None: h0 = torch.zeros((self.num_layers, x.size(0), self.d_model), device=device)
        else: h0 = h
        # print(x.shape, h0.shape)
        out, h = self.rnn(x, h0)
        # out = out[:, -1, :] # out: (n, 128)
        out = self.fc(out) # out: (n, 10)
        return out, h


hidden_size = 128
num_layers = 2
input_size = num_classes = train_dataset.vocab_size#65

model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)
# print(model)
print(sum(p.numel() for p in model.parameters() if p.requires_grad)) # 19683


4960258


In [ ]:
# @title RNN norm
import torch
import torch.nn as nn
device = "cuda" if torch.cuda.is_available() else "cpu"

class RNN(nn.Module):
    def __init__(self, in_dim, d_model, out_dim=None, num_layers=1, batch_first=True, dropout=0.3):
        super().__init__()
        if out_dim is None: out_dim = in_dim
        self.in_dim, self.d_model, self.out_dim, self.num_layers = in_dim, d_model, out_dim, num_layers
        self.fcin = nn.Linear(in_dim, d_model, bias=False)
        self.bn = nn.BatchNorm1d(d_model)
        self.normin = nn.RMSNorm(d_model)
        # self.normout = nn.RMSNorm(d_model)
        self.act = nn.GELU() # GELU SiLU
        self.rnn = nn.GRU(d_model, d_model, num_layers=num_layers, batch_first=batch_first, dropout=dropout)
        self.rnn = nn.GRU(in_dim, d_model, num_layers=num_layers, batch_first=batch_first, dropout=dropout)
        # self.drop = nn.Dropout(.2)
        # self.fc = nn.Linear(d_model, d_model, bias=False)
        self.fc = nn.Linear(d_model, out_dim)

        self.emb = nn.Embedding(in_dim, d_model)
        # self.d_model = d_model
        # self.fc = nn.Linear(d_model, vocab_size)
        # for p in self.parameters():
        #     if p.dim() > 1:
        #         nn.init.xavier_uniform_(p)


    def forward(self, x, h0=None): # [batch_size, seq_len, in_dim]
        x = self.emb(x)
        if h0 is None: h0 = torch.zeros((self.num_layers, x.size(0), self.d_model), device=device)

        # x = self.fcin(x)
        # # # x = self.drop(x)
        # # # x = self.normin(x)
        # x = self.act(x)
        # h0 = self.normin(h0)
        # print("rnn fwd", x.shape, self.in_dim)
        out, h0 = self.rnn(x, h0) # [batch_size, seq_len, d_model], [num_layers, batch_size, d_model]
        # out = out[:, -1, :] # [batch_size, d_model]
        # out = self.drop(out)
        out = self.fc(out) # [batch_size, seq_len, out_dim]
        # out = self.normout(out)
        # out = self.act(out)
        return out, h0


        # # print(x.shape, h0.shape)
        # out, h0 = self.rnn(x, h0)
        # # out = out[:, -1, :] # out: (n, 128)
        # out = self.fc(out) # out: (n, 10)
        # return out, h0


hidden_size = 128
num_layers = 2
input_size = num_classes = train_dataset.vocab_size#65

model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)
# print(model)
print(sum(p.numel() for p in model.parameters() if p.requires_grad)) # 19683



In [ ]:
# @title jepa
# https://openreview.net/pdf?id=BZ5a1r-kVsf
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
scaler = torch.cuda.amp.GradScaler()

def off_diagonal(x):
    n, m = x.shape
    assert n == m
    return x.flatten()[:-1].view(n - 1, n + 1)[:, 1:].flatten()

class JEPA(nn.Module):
    def __init__(self, in_dim, d_model, dim_z, dim_v):
        super().__init__()
        self.enc = nn.Sequential(
            nn.Linear(in_dim, d_model), nn.ReLU(), # ReLU GELU SiLU
            nn.Linear(d_model, d_model), nn.ReLU(),
            nn.Linear(d_model, d_model), nn.ReLU(),
            )
        self.pred = RNN(d_model+dim_z, d_model, d_model)
        self.pred.emb = nn.Linear(d_model+dim_z, d_model)
        # self.pred.emb = nn.Identity()
        self.exp = nn.Sequential(
            nn.Linear(d_model, dim_v), nn.GELU(), # ReLU GELU SiLU
            nn.Linear(dim_v, dim_v), nn.GELU(),
            nn.Linear(dim_v, dim_v),# nn.ReLU(),
            )
        self.d_model = d_model
        self.sim_coeff=25. # 10.0 # 25.0 # λ
        self.std_coeff=25. # 1.0 # 25.0 # µ
        self.cov_coeff=1. # 25.0 # 1.0 # ν

    def v_creg(self, x): # vx [batch_size, d_model]
        x = x - x.mean(dim=0)
        std_x = torch.sqrt(x.var(dim=0) + 0.0001) #ϵ=0.0001
        std_loss = torch.mean(F.relu(1 - std_x)) / 2
        batch_size, num_features = x.shape
        cov_x = (x.T @ x) / (batch_size - 1) #C(Z)
        cov_loss = off_diagonal(cov_x).pow_(2).sum().div(num_features)
        # return self.std_coeff * std_loss, self.cov_coeff * cov_loss
        return std_loss, cov_loss


<ipython-input-18-62c76dbd7373>:10: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()


In [ ]:
# @title agent gru
import torch
import torch.nn as nn
import torch.nn.functional as F
device = "cuda" if torch.cuda.is_available() else "cpu"
import torchvision.transforms as transforms
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
scaler = torch.cuda.amp.GradScaler()

from matplotlib import pyplot as plt
def imshow(img):
    img = img / 2 + 0.5  # unnormalize
    npimg = img.numpy()
    plt.figure(figsize=(4, 4))
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

def ste_argmax(x, temp=1.0):
    x_soft = F.softmax(x / temp, dim=-1)
    x_hard = torch.zeros_like(x)
    x_hard.scatter_(-1, x_soft.argmax(dim=-1, keepdim=True), 1.0)
    x_hard = (x_hard - x_soft).detach() + x_soft
    return x_hard # one hot

class Agent(nn.Module):
    def __init__(self, in_dim=256, d_model=256, dim_z=8, dim_v=512, vocab_size=None):
        super().__init__()
        self.in_dim, self.d_model, self.dim_z, self.dim_v = in_dim, d_model, dim_z, dim_v
        self.jepa = JEPA(in_dim, d_model, dim_z, dim_v)
        # self.tcost = TCost((1+self.jepa.pred.num_layers)*d_model)
        self.emb = torch.nn.Embedding(vocab_size, in_dim)#, max_norm=1.)
        self.jepa.sim_coeff=10. # 10 # λ repr Invariance reconstruction, ->0 slowly
        self.jepa.std_coeff=1. # 1 # µ std Variance
        self.jepa.cov_coeff=1. # 1 # ν cov Covariance
        self.closs_coeff=10. # 3
        # self.zloss_coeff=0. # 10 1
        self.reset()
        # if out_dim is None: out_dim = in_dim
        self.fc = nn.Linear(d_model, vocab_size)

    def reset(self):
        self.h0 = torch.zeros((self.jepa.pred.num_layers, 1, self.d_model), device=device) # [num_layers, batch, d_model]
        # self.lx, self.lz = torch.empty((0,self.dim_a),device=device), torch.empty((0,self.dim_z),device=device) # [T,dim_az]
        self.lz = torch.empty((0,self.dim_z),device=device) # [T,dim_z]
        # self.sx = self.jepa.enc(torch.zeros((1, *self.in_dim))) # [1,d_model]
        self.sx = self.jepa.enc(torch.zeros((1, self.in_dim))) # [1,d_model]

    def forward(self, x):
        self.train()
        # for batch, (x,y) in enumerate(dataloader): # collate: [seq_len, batch_length], default: [batch_size, seq_len]
        h0 = torch.zeros((self.jepa.pred.num_layers, batch_size, self.d_model), device=device) # [num_layers, batch, d_model]
        # sy_ = self.jepa.enc(torch.zeros((batch_size, *self.in_dim), device=device)).unsqueeze(1) # [batch_size, 1, d_model]
        sy_ = self.jepa.enc(torch.zeros((batch_size, self.in_dim), device=device)).unsqueeze(1) # [batch_size, 1, d_model]
        x = x.to(device)
        with torch.cuda.amp.autocast():
            st = self.emb(st)
            lsy = self.jepa.enc(st.flatten(end_dim=1)).unflatten(0, (batch_size, -1)) # [batch_size, bptt, d_model]


    # def forward(self, lstate, laction=None, k=1): # live run in env # np (64, 64, 3)
    #     self.eval()
    #     if len(self.la)>1 or laction!=None:
    #         self.update_h0(lstate, laction)
    #     else:
    #         with torch.no_grad():
    #             self.sx = self.jepa.enc(lstate[-1])#.unsqueeze(0)
    #             # self.icost.update(sx)
    #     lact, lh0, lx, lz = self.search(self.sx, T=6, h0=self.h0) # [T], [T, num_layers, d_model], [T, dim_a], [T, dim_z]
    #     act = lact.cpu()[:k].tolist()
    #     self.la, self.lx, self.lz = lact, lx, lz
    #     return act

    # def update_h0(self, lstate, laction=None): # live run in env # np (64, 64, 3)
    #     with torch.no_grad():
    #         with torch.cuda.amp.autocast():
    #             lsy = self.jepa.enc(torch.cat(lstate, dim=0))
    #             # self.icost.update(sx)
    #             seq_len = len(lstate)
    #             if laction!=None: self.la = torch.cat([torch.tensor(laction, device=device), self.la[len(laction):]], dim=-1)
    #             la = self.emb(self.la[:seq_len])

    #     batch = 64 # 16
    #     lz = nn.Parameter(torch.zeros((batch, seq_len, self.dim_z),device=device))
    #     torch.nn.init.normal_(lz, mean=0., std=.3/lz.shape[-1]**0.5) # torch.nn.init.xavier_normal_(lz) # xavier_normal_ xavier_uniform_
    #     # optim_z = torch.optim.SGD([lz], lr=1e0) # 1e0
    #     optim_z = torch.optim.SGD([lz], lr=1e0, momentum=.3) # 1e0,.3
    #     # optim_z = torch.optim.AdamW([lz], 1e-3, (0.9, 0.999)) # 1e0 ; 3e-2 1e-1
    #     # optim_z = torch.optim.AdamW([lz], 1e-1, (0.1, 0.5)) # 1e-3
    #     lsy = lsy.detach() # [T, d_model]
    #     lsx = torch.cat([self.sx, lsy[:-1]], dim=0) # [T, dim_a]
    #     lsy, lsx = lsy.repeat(batch,1,1), lsx.repeat(batch,1,1) # [batch, T, d_model], [batch, T, dim_a]
    #     # out_ = lsy - lsx

    #     # print("update_h0 lz", lz[0][0].data)
    #     self.jepa.pred.train()
    #     for i in range(5): # 1?
    #         sxz = torch.cat([lsx, lz], dim=-1)#.unsqueeze(0) # [batch, seq_len, d_model+dim_a+dim_z]
    #         with torch.cuda.amp.autocast(): # with torch.amp.autocast('cuda'):
    #             out, h0 = self.jepa.pred(sxz, self.h0.detach().repeat(1,batch,1)) # [batch,seq_len,d_model], [num_layers,batch,d_model]
    #             loss = ((lsy-out)**2).flatten(1).mean(-1) # [batch,seq_len,d_model] -> [batch]
    #             # loss = ((out_-out)**2).flatten(1).mean(-1) # [batch,seq_len,d_model] -> [batch]
    #         loss.sum().backward()
    #         optim_z.step()
    #         optim_z.zero_grad()
    #         # print("update_h0 loss, lz",i,loss[0].item(), lz[0][0].data)
    #         print("update_h0 loss",i,loss[0].item())
    #         with torch.no_grad(): lz /= torch.norm(lz, dim=-1).unsqueeze(-1).clamp_(min=1)
    #     # print("loss",loss.data)
    #     idx = torch.argmax(loss, dim=0) # loss [batch*batch_, T] -> [batch_]
    #     self.h0 = h0[:,idx]
    #     self.sx = lsx[idx][-1].unsqueeze(0)
    #     self.lx, self.lz = self.lx[seq_len:], self.lz[seq_len:] # [T], [T, dim_z]
    #     return self.h0


    def argm_s(self, sx, x, h0, zz=None): # [1, d_model], [batch_, T, dim_a], [num_layers, 1, d_model], zz[batch_,T, dim_z] # batch argm z for search
        batch_, T, _ = x.shape
        batch = 16 # 16
        z = nn.Parameter(torch.zeros((batch*batch_, T, self.dim_z),device=device))
        torch.nn.init.normal_(z, mean=0., std=1./z.shape[-1]**0.5) # 1.:norm~1 ; 1.
        optim_z = torch.optim.SGD([z], lr=1e3, momentum=0.999, maximize=True) # 3e3
        # optim_z = torch.optim.AdamW([z], 1e-1, (0.1, 0.5), maximize=True) # 1e-1, (0.1, 0.5)
        with torch.no_grad():
            z[:,:self.lz.shape[0]] = self.lz[:T].unsqueeze(0).repeat(batch*batch_,1,1) # [batch*batch_, seq_len, dim_z]
            if zz != None: z[:batch_]=zz #z[::batch]
        sx, h0 = sx.detach(), h0.detach() # [1, d_model], [num_layers, 1, d_model]
        x = x.detach().repeat(batch,1,1) # [batch, T, dim_a]
        # print("argm_s", z[0][0].squeeze())
        for i in range(5): # 2 5
            loss, lh0 = self.rnn_pred(sx, x, z, h0) # snap x to act emb
            loss.sum().backward() # [batch, T]
            optim_z.step()
            optim_z.zero_grad()
            with torch.no_grad(): z /= torch.norm(z, dim=-1).unsqueeze(-1).clamp_(min=1) # z.clamp_(min=-1, max=1)
            # print(i, "argm_s z", z[0][0].squeeze().data)
            print(i, "argm_s loss", loss.sum(-1)[0].data)
            # # print(torch.norm(z, dim=-1)) # all 1
        idx = torch.argmax(loss.sum(-1).unflatten(0, (batch,batch_)), dim=0) # loss [batch*batch_, T] -> [batch_]
        # print("argm_s loss[idx]", loss[idx].sum().item())
        return z.unflatten(0, (batch,batch_))[idx, torch.arange(batch_)]


    # def search(self, sx, T=6, h0=None):
    #     batch = 16 # 16
    #     x = nn.Parameter(torch.zeros((batch, T, self.dim_a),device=device))
    #     torch.nn.init.normal_(x, mean=0., std=.3/x.shape[-1]**0.5) # .3
    #     # optim_x = torch.optim.SGD([x], lr=1e1, momentum=.7) # 1e-1,1e-0,1e4 ; 1e2 ; 1e1,.7
    #     optim_x = torch.optim.AdamW([x], 1e-1, (0.1, 0.999)) # 1e-1 (0.1, 0.999)
    #     with torch.no_grad(): x[:,:self.lx.shape[0]] = self.lx.repeat(batch,1,1)[:,:T] # [seq_len, dim_az]
    #     sx, h0 = sx.detach(), h0.detach() # [1, d_model], [num_layers, 1, d_model]
    #     # print("search x",x[0][0].squeeze().data)
    #     # z=None
    #     z = torch.zeros((batch, T, self.dim_z),device=device)
    #     for i in range(100):
    #         dist = torch.norm(self.emb.weight.data[None,None,...] - x.unsqueeze(-2), dim=-1) # [1,1,act_space,emb_dim], [batch,T,1,emb_dim] -> [batch,T,act_space]
    #         x_ = ste_argmax(-dist) @ self.emb.weight.data # [batch, T, dim_a]
    #         # z = self.argm_s(sx, x_,h0,z) # [batch,T, dim_z]
    #         loss, lh0 = self.rnn_pred(sx, x_, z, h0) # snap x to act emb
    #         # print(i, "search x", x[0][0].squeeze().data)
    #         print(i, "search loss", loss.sum(-1)[0].data)

    #         # reg

    #         if i>=6: break # 5 10
    #         loss.sum().backward()
    #         optim_x.step()
    #         optim_x.zero_grad()
    #         with torch.no_grad(): x /= torch.norm(x, dim=-1).unsqueeze(-1).clamp_(min=1)
    #     idx = torch.argmin(loss.sum(-1)) # loss [batch, T]
    #     dist = torch.norm(self.emb.weight.data.unsqueeze(0) - x.unsqueeze(-2)[idx], dim=-1) # [1,act_space,emb_dim], [T,1,emb_dim] -> [T,act_space]
    #     lact = torch.argmin(dist, dim=-1) # [T]
    #     # print(lact.data)
    #     # print("search loss[idx]",loss[idx].sum().item())
    #     return lact, lh0[:,:,idx], x.data[idx], z[idx] # [batch,T], [T, num_layers, batch, d_model], [batch,T, dim_a], [batch,T, dim_z]


    # def rnn_pred(self, sx, la, lz, h0, gamma=0.9): # 0.95 [1, d_model], [batch, T, dim_a/z], [num_layers,1, d_model]
    #     self.jepa.pred.train()
    #     batch, seq_len, _ = la.shape
    #     lsx, lh0 = self.rnn_it(sx, la, lz, h0)
    #     icost = 0#*0.0005*self.icost.boredom(lsx, linsx=None) # + self.icost(sx)
    #     syh0 = torch.cat([lsx, lh0.permute(2,0,1,3).flatten(2)], dim=-1).flatten(end_dim=1) # [batch_size,T,d_model], [T,num_layers,batch_size,d_model] -> [batch_size*bptt, (1+num_layers)*d_model]
    #     tcost = -self.tcost(syh0).unflatten(0, (batch, seq_len)).squeeze(-1)
    #     c = (tcost + icost)*gamma**torch.arange(seq_len, device=device)
    #     # if len(c.shape) == 1: print("rnn_pred c", [f'{cc.item():g}' for cc in c.squeeze(0)]) # print(f'{cc:6f}')
    #     if len(tcost.shape) == 1: print("rnn_pred tcost", [f'{cc.item():g}' for cc in tcost.squeeze(0)]) # print(f'{cc:6f}')
    #     return c, lh0

    def rnn_it(self, sx, lz, h0): # 0.95 [1, d_model], [batch, T, dim_z], [num_layers,1, d_model]
        self.jepa.pred.train()
        batch, seq_len, _ = lz.shape
        batch_ = batch//sx.shape[0]
        sx, h0 = sx.repeat(batch_, 1), h0.repeat(1, batch_, 1)
        lsx = torch.empty((batch, 0, self.d_model), device=device) # [batch_size, T, d_model]
        lh0 = torch.empty((0,)+h0.shape, device=device)
        for t in range(seq_len):
            z = lz[:,t] # [1, dim_z]
            sxz = torch.cat([sx, z], dim=-1).unsqueeze(1) # [batch, d_model+dim_z]
            with torch.amp.autocast('cuda'):
                out, h0 = self.jepa.pred(sxz, h0) # [batch,seq_len,d_model], [num_layers,batch,d_model]
            # sx = sx + out.squeeze(1) # [batch,seq_len,d_model] # h0 = h0 +
            sx = out.squeeze(1) # [batch,1,d_model]
            lsx = torch.cat([lsx, sx.unsqueeze(1)], dim=1) # [batch, T, d_model]
            lh0 = torch.cat([lh0, h0.unsqueeze(0)], dim=0) # [seq_len, num_layers, batch, d_model]
        return lsx, lh0

    def argm(self, lsy, sy, h0): # best case z for train
        batch_size, bptt, _ = lsy.shape # [batch_size, bptt, d_model]
        batch = 64 # 64
        lsy = lsy.repeat(batch,1,1) # [batch*batch_size, bptt, d_model], [batch*batch_size, d_model, dim_a], [batch*batch_size, bptt]
        lz = nn.Parameter(torch.zeros((batch*batch_size, bptt, self.dim_z), device=device))
        torch.nn.init.normal_(lz, mean=0., std=.25/lz.shape[-1]**0.5) # .25?
        # optim = torch.optim.SGD([lz], lr=1e2, momentum=.5) # 1e-2 1e3 1e2 1e1 1e2,.5
        optim = torch.optim.AdamW([lz], 1e-2, (0.1, 0.9)) # 1e-1, (0.1, 0.5)
        lsy = lsy.detach()
        for i in range(5): # 3 10
            sy_, h0_ = sy.detach(), h0.detach()
            lsy_, lh0 = self.rnn_it(sy_, lz, h0_)
            # repr_loss = F.mse_loss(lsy, lsy_)
            repr_loss = ((lsy-lsy_)**2).unflatten(0, (batch,batch_size)).flatten(1).mean(-1)
            syh0 = torch.cat([lsy, lh0.permute(2,0,1,3).flatten(2)], dim=-1).flatten(end_dim=1) # [batch*batch_size,bptt,d_model], [bptt,num_layers,batch*batch_size,d_model] -> [batch*batch_size*bptt, (1+num_layers)*d_model]
            # z_loss = torch.abs(lz).sum() # z_loss = torch.norm(z)
            # print("argm lz[0]", i, lz[0][0].data)
            # # print(i, repr_loss[0].item())
            # print(repr_loss.data)
            cost = self.jepa.sim_coeff * repr_loss # + self.zloss_coeff * z_loss
            cost.sum().backward()
            optim.step()
            optim.zero_grad()
            with torch.no_grad(): lz /= torch.norm(lz, dim=-1).unsqueeze(-1).clamp_(min=1)
        # print(repr_loss.data)
        idx = torch.argmin(cost)
        return lz.unflatten(0, (batch,batch_size))[idx].squeeze(0).detach()

    def train_jepa(self, dataloader, optim, bptt=25): #32
        self.train()
        for batch, (x,y) in enumerate(dataloader): # collate: [seq_len, batch_length], default: [batch_size, seq_len]
            h0 = torch.zeros((self.jepa.pred.num_layers, batch_size, self.d_model), device=device) # [num_layers, batch, d_model]
            # sy_ = self.jepa.enc(torch.zeros((batch_size, *self.in_dim), device=device)).unsqueeze(1) # [batch_size, 1, d_model]
            sy_ = self.jepa.enc(torch.zeros((batch_size, self.in_dim), device=device)).unsqueeze(1) # [batch_size, 1, d_model]
            x = x.to(device)
            for st in torch.split(x, bptt, dim=1):
                with torch.cuda.amp.autocast():
                    st = self.emb(st)
                    lsy = self.jepa.enc(st.flatten(end_dim=1)).unflatten(0, (batch_size, -1)) # [batch_size, bptt, d_model]
                    lz = self.argm(lsy, sy_.squeeze(1), h0) # [batch_size, bptt, d_model],
                    # with torch.no_grad(): lz.mul_(torch.rand_like(lz).uniform_(0.5)).mul_((torch.rand_like(lz)>0.1).bool()) # dropout without scaling
                    with torch.no_grad(): lz.mul_(torch.rand_like(lz).uniform_(0)).mul_((torch.rand_like(lz)>0.5).bool()) # dropout without scaling
                    lsy_, lh0 = self.rnn_it(sy_.squeeze(1), lz, h0)
                    repr_loss = F.mse_loss(lsy, lsy_) # [batch_size, bptt, d_model]
                    std_loss, cov_loss = self.jepa.v_creg(self.jepa.exp(lsy.flatten(end_dim=1)))
                    loss = self.jepa.sim_coeff * repr_loss + self.jepa.std_coeff * std_loss + self.jepa.cov_coeff * cov_loss
                    # syh0 = torch.cat([lsy, lh0.permute(2,0,1,3).flatten(2)], dim=-1).flatten(end_dim=1) # [batch_size,bptt,d_model], [bptt,num_layers,batch_size,d_model] -> [batch_size*bptt, (1+num_layers)*d_model] # not lsy_, else unstable

                    # reprloss = ((lsy-lsy_)**2).mean(-1) # [batch_size, bptt]
                    # print("reprloss",reprloss[0])

                # torch.norm(lsy-torch.cat([sy_,lsy[:-1]], dim=1), dim=-1) # -(z*torch.log(z)).sum(-1) # Shannon entropy archive.is/CaYrq
                # prob = F.softmax(output, dim=-1)
                # entropy = -torch.sum(prob * torch.log(prob + 1e-5), dim=-1)

                # print("norm", torch.norm(sy, dim=-1)[0].item())
                norm = torch.norm(lsy[0][0], dim=-1).item()
                z_norm = torch.norm(lz[0][-1], dim=-1)
                print("repr, std, cov, z, norm", repr_loss.item(), std_loss.item(), cov_loss.item(), z_norm.item(), norm)
                scaler.scale(loss).backward()
                # torch.nn.utils.clip_grad_norm_(self.parameters(), 1)
                scaler.step(optim)
                scaler.update()
                optim.zero_grad()
                sy_, h0 = sy_.detach(), h0.detach()
                try: wandb.log({"repr": repr_loss.item(), "std": std_loss.item(), "cov": cov_loss.item()})
                except: pass



vocab_size = train_dataset.vocab_size
model = Agent(d_model=256, vocab_size=vocab_size).to(device)
# agent = torch.compile(Agent(d_model=256), mode='max-autotune').to(device)

optim = torch.optim.AdamW(model.parameters(), 1e-3, (0.9, 0.999)) # lr = 1e-4 #3e-4
# optim = torch.optim.AdamW(agent.parameters(), 1e-4, (0.9, 0.95)) # lr = 1e-4
# !pip show torch triton
# # !pip install --upgrade torch
# !pip install --upgrade triton

# print(sum(p.numel() for p in agent.parameters() if p.requires_grad)) # 1lyr:2727982, 2lyr:4401710
# print(sum(p.numel() for p in agent.jepa.enc.parameters() if p.requires_grad)) # 1278976
# print(sum(p.numel() for p in agent.jepa.pred.parameters() if p.requires_grad)) # 1lyr:397824, 2lyr:792576
# print(sum(p.numel() for p in agent.tcost.parameters() if p.requires_grad)) # 197633


<ipython-input-19-6c8913fde5c3>:9: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()


In [ ]:
# @title HRNN pytorch
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = "cuda" if torch.cuda.is_available() else "cpu"

import math

# Fully connected neural network with one hidden layer
class RNN(nn.Module):
    def __init__(self, in_dim, d_model, out_dim=None, num_layers=1):
        super().__init__()
        if out_dim is None: out_dim = in_dim
        self.num_layers = num_layers
        self.d_model = d_model
        self.rnn = nn.GRU(d_model, d_model, num_layers, batch_first=True)
        self.fc = nn.Linear(d_model, out_dim)
        self.emb = nn.Embedding(in_dim, d_model)

    def forward(self, x, h=None): # rnn/gru
        x = self.emb(x)# * math.sqrt(self.d_model)
        if h is None: h0 = torch.zeros((self.num_layers, x.size(0), self.d_model), device=device)
        else: h0 = h
        # print(x.shape, h0.shape)
        out, h = self.rnn(x, h0)
        # out = out[:, -1, :] # out: (n, 128)
        out = self.fc(out) # out: (n, 10)
        return out, h






class RNN(nn.Module): # https://github.com/georgeyiasemis/Recurrent-Neural-Networks-from-scratch-using-PyTorch/blob/main/rnnmodels.py
    def __init__(self, in_dim, d_model, num_layers, out_dim):
        super().__init__()
        self.num_layers = num_layers
        self.d_model = d_model
        self.tok_emb = nn.Embedding(in_dim, d_model)
        self.rnn = nn.ModuleList([GRUCell(d_model, d_model) for _ in range(num_layers)])
        self.fc = nn.Linear(d_model, out_dim)



    def forward(self, x, h0=None):
        batch_size, seq_len = x.shape
        x = self.tok_emb(x) #[batch_size, seq_len, d_model]
        if h0==None: h0 = torch.zeros(self.num_layers, batch_size, self.d_model, device=device)
        ht = list(torch.split(h0, 1))
        out = []
        for t in range(seq_len):
            for i, layer in enumerate(self.rnn):
                if i==0: ht[i] = layer(x[:,t], h0[i])
                else: ht[i] = layer(ht[i-1], h0[i])
            out.append(ht[-1]) # seq_len * [batch_size, d_model]
            h0 = ht
        ht = torch.stack(ht, dim=0)
        out = torch.stack(out, dim=1) # [batch_size, seq_len, d_model]
        out = self.fc(out) # out: (n, 10)
        return out, ht # rnn/gru


        # torch.norm(lsy-torch.cat([sy_,lsy[:-1]], dim=1), dim=-1) #
        # prob = F.softmax(output, dim=-1)
        # entropy = -torch.sum(prob * torch.log(prob + 1e-5), dim=-1)
        -(z * torch.log(z + 1e-5)).sum(-1) # Shannon entropy archive.is/CaYrq










hidden_size = 128
num_layers = 2
input_size = num_classes = train_dataset.vocab_size#65

model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)
# print(model)
print(sum(p.numel() for p in model.parameters() if p.requires_grad)) # 19683


4960258


In [ ]:
# @title wandb
# https://docs.wandb.ai/quickstart
!pip install wandb
import wandb
wandb.login() # 487a2109e55dce4e13fc70681781de9f50f27be7
run = wandb.init(
    project="tiny_gpt",
    config={
        "model": "adam 1e-3",
        "optim": "adam",
        # "learning_rate": 5,
    })


In [ ]:
# @title train test generate
import torch
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
scaler = torch.cuda.amp.GradScaler()

def strain(model, dataloader, optimizer, loss_fn, scheduler=None): # train function with automatic mixed precision
    model.train()
    total_loss = 0
    for batch, (data, targets) in enumerate(dataloader):
        data, targets = data.to(device), targets.to(device)
        with torch.cuda.amp.autocast():
            logits = model(data)
            loss = loss_fn(logits.reshape(-1, logits.size(-1)), targets.flatten()) # [512, 128, 283], [512, 128]
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        # torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        scaler.step(optimizer)
        scaler.update()
        # if scheduler is not None: scheduler.step()
        # print("strain",loss.item())
        total_loss += loss.item()
        try: wandb.log({"train loss": loss.item()/len(targets)})
        except NameError: pass
    return total_loss / len(dataloader)


from tqdm import tqdm
def train(loader, model, loss_fn, optimizer):
    model.train()
    total_loss = 0
    pbar = tqdm(enumerate(loader), total = len(loader))
    for it, (x, y) in pbar:
    # for it, (x, y) in enumerate(loader):
        x, y = x.to(device), y.to(device)
        logits, _ = model(x)
        # print("in train:",logits.shape,y.shape)
        # loss = loss_fn(logits.reshape(-1, logits.size(-1)), y[:,-1]) # [batch, vocab_size], [batch]
        loss = loss_fn(logits.flatten(end_dim=-2), y.flatten()) # [batch*seq_len, vocab_size], [batch*seq_len]
        optimizer.zero_grad()
        loss.backward()
        # torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
        optimizer.step()
        total_loss+=loss.item()
        try: wandb.log({"train loss": loss.item()/len(y)})
        except NameError: pass
        pbar.set_description(f"epoch {epoch + 1} iter {it}: train loss {loss.item():.5f}.")
    return total_loss / len(loader)

def test(loader, model, loss_fn):
    model.eval()
    total_loss = 0
    for it, (x, y) in enumerate(loader):
        x, y = x.to(device), y.to(device)
        with torch.no_grad():
            logits, _ = model(x)
        loss = loss_fn(logits.flatten(end_dim=-2), y.flatten()) # [batch*seq_len, vocab_size], [batch*seq_len]
        total_loss+=loss.item()
    return total_loss / len(loader)


from torch.nn import functional as F
def generate(model, context, max_steps = 64):
    x = torch.tensor([train_dataset.stoi.get(c) for c in context], device=device)
    model.eval()
    for n in range(max_steps):
        if x.shape[0] <= seq_len: x_bar = x
        else: x_bar = x[-seq_len:]
        output = model(x_bar.unsqueeze(0)) # [1, len(contex)+, vocab_size=283] float
        output = output[-1,:]
        output = F.softmax(output, dim = -1) # vocab_size to char
        ix = torch.multinomial(output, num_samples = 1) # rand sample by output distribution
        x = torch.cat((x, ix.flatten()))
    # print([int(i) for i in x])
    completion = ''.join([train_dataset.itos[int(i)] for i in x])
    return completion


import torch
from torch.nn import functional as F
def generate(model, context, max_steps = 64, temperature=1):
    # x = torch.tensor([train_dataset.stoi.get(c) for c in context], device=device).unsqueeze(0)
    x=ix = torch.tensor([train_dataset.stoi.get(c) for c in context], device=device).unsqueeze(0)
    model.eval()
    hidden=None
    with torch.no_grad():
        for n in range(max_steps):
            # output, hidden = model(x, hidden)
            output, hidden = model(ix, hidden)
            if type(hidden) == torch.Tensor: hidden=hidden[:, -1, :].unsqueeze(1) # RNN/GRU
            elif type(hidden) == tuple: hidden = hidden[0][:, -1, :].unsqueeze(1), hidden[1][:, -1, :].unsqueeze(1) # LSTM
            output = output[:, -1, :] # get logit for last character
            output = output/temperature
            output = F.softmax(output, dim = -1) # vocab_size to char
            ix = torch.multinomial(output, num_samples = 1) # rand sample by output distribution
            x = torch.cat((x, ix),1)
        completion = ''.join([train_dataset.itos[int(i)] for i in x.flatten()])
        return completion

# out=generate(model, "A wi")
# print(out)



<ipython-input-11-032d8ca28759>:5: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()


In [ ]:
# @title run
# AdamW 1e-4 1e-3 # sgd 1e-3
optimizer = torch.optim.AdamW(model.parameters(), 1e-3, (0.9, 0.95)) # 1e-4 #3e-4
# optimizer = torch.optim.SGD(model.parameters(), lr=1e-3) # 1e-3
criterion = nn.CrossEntropyLoss()
# print("lr: ", optimizer.param_groups[0]['lr'])
# optimizer.param_groups[0]['lr']=1e-3

for epoch in range(30):
    # train_loss = strain(model, train_loader, optimizer, criterion, scheduler=None)
    # train_loss = train(train_loader, model, criterion, optimizer)
    model.train_jepa(train_loader, optimizer)

    # test_loss = test(test_loader, model, criterion)
    # print('Test Loss:', test_loss)
    print(generate(model, "this is what "))


<ipython-input-19-6c8913fde5c3>:232: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


repr, std, cov, z, norm 0.007034624926745892 0.49490612745285034 6.573959709932353e-10 0.24961885809898376 1.0526517629623413
repr, std, cov, z, norm 0.002107883570715785 0.494915634393692 6.869880220250479e-10 0.07965850830078125 0.8051280975341797
repr, std, cov, z, norm 0.0009434848325327039 0.4949428141117096 3.2224872770214574e-10 0.11793156713247299 0.4428272247314453
repr, std, cov, z, norm 0.000577109691221267 0.4949592649936676 1.5591533619740972e-10 0.1225781962275505 0.39861851930618286
repr, std, cov, z, norm 0.000411276298109442 0.4949593245983124 2.2416371536770896e-10 0.1524714082479477 0.3415570855140686
repr, std, cov, z, norm 0.00031864841002970934 0.4949473738670349 6.278619846256106e-10 0.11882088333368301 0.38344061374664307
repr, std, cov, z, norm 0.0002586319169495255 0.49492788314819336 1.786807590775652e-09 0.04149853438138962 0.3138118088245392
repr, std, cov, z, norm 0.00020672280516009778 0.4948912262916565 5.458593577145621e-09 0.10362884402275085 0.3403543

KeyboardInterrupt: 

In [ ]:
# @title inference
context = "this is what "
#context = 'There are many things about horses that have been discovered in recent'
completion = generate(model, context,200)
print(completion)
# print(generate(model, "this is what "))


NotImplementedError: Module [Agent] is missing the required "forward" function

In [ ]:
# @title save/load

# with open(folder+'buffergo.pkl', 'rb') as f: buffer = pickle.load(f)
# with open('buffergo.pkl', 'rb') as f: buffer = pickle.load(f)


# from google.colab import drive
# drive.mount('/content/drive')
# folder='/content/drive/MyDrive/jepa/'
# import pickle

# path = "/content/drive/MyDrive/frame/RNNS.pth"
path = "/hjepa.pth"
torch.save(model.state_dict(), path)
# model.load_state_dict(torch.load(path, map_location=device))

# modelsd, optimsd = torch.load(folder+'agentoptim.pkl', map_location=device).values()
# # modelsd, optimsd = torch.load('agentoptim.pkl', map_location=device).values()
# agent.load_state_dict(modelsd, strict=False)
# optim.load_state_dict(optimsd)



In [ ]:
torch.cuda.empty_cache()

In [ ]:
# !pip install line_profiler
%load_ext line_profiler
%lprun -f model.selective_scan() train(train_loader, model, criterion, optimizer)
    # train_loss = train(train_loader, model, criterion, optimizer)


The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


UsageError: Could not find module model.selective_scan().
AttributeError: 'MambaLM' object has no attribute 'selective_scan'


## Mamba

In [ ]:
# @title Mamba_S6
# https://github.com/johnma2006/mamba-minimal/blob/master/model.py
"""[1] Mamba: Linear-Time Sequence Modeling with Selective State Spaces (Albert Gu and Tri Dao) https://arxiv.org/abs/2312.00752
    [2] The Annotated S4 (Sasha Rush and Sidd Karamcheti) https://srush.github.io/annotated-s4

    b: batch size                       (`B` in Mamba paper [1] Algorithm 2)
    l: sequence length                  (`L` in [1] Algorithm 2)
    d or d_model: hidden dim
    n or d_state: latent state dim      (`N` in [1] Algorithm 2)
    d_in or d_inner: d * expansion factor         (`D` in [1] Algorithm 2) ; expansion factor (`E` in [1] Section 3.4)
    A, B, C, D: state space parameters  (See any state space representation formula)
                                        (B, C are input-dependent (aka selective, a key innovation in Mamba); A, D are not)
    Δ or delta: input-dependent step size
    dt_rank: rank of Δ                  (See [1] Section 3.6 "Parameterization of ∆")
"""
import torch
import torch.nn as nn
import torch.nn.functional as F
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class Mamba(nn.Module):
    def __init__(self, d_model = 256, d_state = 16, d_inner = 512):
        super().__init__()
        self.d_model, self.d_state, self.d_inner = d_model, d_state, d_inner
        n_layer = 3
        self.layers = nn.ModuleList([ResidualBlock(d_model, d_state, d_inner) for _ in range(n_layer)])
        self.norm = nn.RMSNorm(d_model)

    def forward(self, x, h=None): # (b, l, d)
        if h is None: h = torch.zeros((x.shape[0], self.d_inner, self.d_state), device=device)
        for layer in self.layers:
            x, h = layer(x, h)
        x = self.norm(x)
        return x, h

class MambaLM(nn.Module):
    def __init__(self, vocab_size, d_model = 128, d_state = 16, d_inner = 128):
        super().__init__()
        self.d_model, self.d_state, self.d_inner = d_model, d_state, d_inner
        n_layer = 2#3
        self.tok_emb = nn.Embedding(vocab_size, d_model)
        self.layers = nn.ModuleList([ResidualBlock(d_model, d_state, d_inner) for _ in range(n_layer)])
        self.norm_f = nn.RMSNorm(d_model)
        self.fc = nn.Linear(d_model, vocab_size)

    def forward(self, x, h=None): # (b, l, d)
        if h is None: h = torch.zeros((x.shape[0], self.d_inner, self.d_state), device=device)
        x = self.tok_emb(x) #[batch_size, seq_len, d_model]
        for layer in self.layers:
            x, h = layer(x, h)
        x = self.norm_f(x)
        x = self.fc(x) # out: (n, 10)
        return x, h


class ResidualBlock(nn.Module):
    def __init__(self, d_model, d_state, d_inner):
        super().__init__()
        self.block = MambaBlock(d_model, d_state, d_inner)
        self.norm = nn.RMSNorm(d_model)

    def forward(self, x, h): # (b, l, d) # Block.forward(), https://github.com/state-spaces/mamba/blob/main/mamba_ssm/modules/mamba_simple.py#L297
        y, h = self.block(self.norm(x), h)
        return x + y, h # (b, l, d)


class MambaBlock(nn.Module):
    def __init__(self, d_model, d_state, d_inner, dt_rank = 16):
        super().__init__()
        self.d_state, self.dt_rank = d_state, dt_rank
        self.in_proj = nn.Linear(d_model, d_inner * 2, bias=False)
        d_conv = 4
        self.conv1d = nn.Conv1d(d_inner, d_inner, d_conv, padding=d_conv-1, groups=d_inner)
        self.x_proj = nn.Linear(d_inner, dt_rank + d_state * 2, bias=False) # takes in `x` and outputs the input-specific Δ, B, C
        self.dt_proj = nn.Linear(dt_rank, d_inner, bias=True) # project Δ from dt_rank to d_in

        A = torch.arange(1, d_state + 1).repeat(d_inner, 1)
        self.A_log = nn.Parameter(torch.log(A))
        self.D = nn.Parameter(torch.ones(d_inner))
        self.out_proj = nn.Linear(d_inner, d_model, bias=False)

    def forward(self, x, h): # (b, l, d) # https://github.com/state-spaces/mamba/blob/main/mamba_ssm/modules/mamba_simple.py#L119
        b, l, d = x.shape
        x_res = self.in_proj(x)  # (b, l, 2 * d_in)
        x, res = torch.chunk(x_res, 2, dim=-1) # [b, l, d_in]
        x = self.conv1d(x.permute(0, 2, 1))[:, :, :l].permute(0, 2, 1) # (b, l, d_in)
        y, h = self.ssm(F.silu(x), h)
        output = self.out_proj(y * F.silu(res))
        return output, h # (b, l, d)


    def ssm(self, x, h): # (b, l, d_in)
        """- Algorithm 2 in Section 3.2 in the Mamba paper [1] - run_SSM(A, B, C, u) in The Annotated S4 [2]
        Official Implementation: mamba_inner_ref(), https://github.com/state-spaces/mamba/blob/main/mamba_ssm/ops/selective_scan_interface.py#L311"""
        A = -torch.exp(self.A_log.float())  # shape (d_in, n)
        # A = -torch.exp(self.A_log)  # shape (d_in, n)
        D = self.D.float()

        # 2: B : (B, L, N) ← sB (x)
        # 3: C : (B, L, N) ← sC (x)
        # 4: Δ : (B, L, D) ← 𝜏Δ(Parameter+𝑠Δ(x)) # 𝑠Δ(𝑥) = Broadcast𝐷 (Linear1(𝑥)), and 𝜏Δ = softplus,
        # 5: A, B : (B, L, D, N) ← discretize(Δ, A, B)
        # 6: y ← SSM(A, B, C) (x)

        B_C_delta = self.x_proj(x)  # (b, l, dt_rank + 2*n)
        B, C, delta = B_C_delta.split([self.d_state, self.d_state, self.dt_rank], dim=-1) # B, C: (b, l, n); delta: (b, l, dt_rank)
        delta = F.softplus(self.dt_proj(delta)) # (b, l, d_in)
        y, h = self.selective_scan(x, delta, A, B, C, D, h)  # This is similar to run_SSM(A, B, C, u) in The Annotated S4 [2]
        return y, h # (b, l, d_in), (b, d_in, n)

    def selective_scan(self, x, delta, A, B, C, D, h): # u: (b, l, d_in); delta: (b, l, d_in); A: (d_in, n); B: (b, l, n); C: (b, l, n); D: (d_in,)
        """- Section 2 State Space Models in the Mamba paper [1] - Algorithm 2 in Section 3.2 in the Mamba paper [1] - run_SSM(A, B, C, u) in The Annotated S4 [2]
        hidden x(t+1) = Ax(t) + Bu(t); input u
        output y(t)   = Cx(t) + Du(t)
        hidden ht = Aht-1 + Bx; input x
        output y = Cht + Dx
        except B and C (and the step size delta, which is used for discretization) are dependent on the input x(t).

        Official Implementation: selective_scan_ref(), https://github.com/state-spaces/mamba/blob/main/mamba_ssm/ops/selective_scan_interface.py#L86
        Note: I refactored some parts out of `selective_scan_ref` out, so the functionality doesn't match exactly."""
        b, l, d_in = x.shape
        # n = A.shape[1]

        # Discretize continuous parameters (A, B)
        # deltaA = torch.exp(torch.einsum('bld,dn->bldn', delta, A)) # [b l d_in n] # - A is discretized using zero-order hold (ZOH) discretization (see Section 2 Equation 4 in the Mamba paper [1])
        # deltaB_x = torch.einsum('bld,bln,bld->bldn', delta, B, x) # [b l d_in n] # - B is discretized using a simplified Euler discretization instead of ZOH. From a discussion with authors: "A is the more important term and the performance doesn't change much with the simplification on B"
        # 64, 100, 128, 16]) torch.Size([100, 8192, 100, 16]
        deltaA = A.repeat(b, l, 1, 1)
        deltaB_x = B.unsqueeze(-2).repeat(1, 1, d_in, 1)
        # print(deltaA.shape, deltaB_x.shape)

        # selective scan (scan_SSM() in The Annotated S4 [2])
        # Note that the below is sequential, while the official implementation does a much faster parallel scan that is additionally hardware-aware (like FlashAttention).
        ys = []
        for i in range(l):
            h = deltaA[:, i] * h + deltaB_x[:, i]
            y = torch.einsum('bdn,bn->bd', h, C[:, i, :]) # b d_in
            ys.append(y)
        y = torch.stack(ys, dim=1)  # shape (b, l, d_in)
        y = y + D * x # y + u * D
        return y, h # (b, l, d_in), (b, d_in, n)


# pred = Mamba()

# batch=4
# seq_len=500
# d_model=256
# x = torch.randn(batch, seq_len, d_model)
# y, h = pred(x)
# # print(x)
# # print(y)
# print(x.shape)
# print(y.shape) # [batch, seq_len, d_model]
# print(h.shape) # [batch, d_inner, d_state]


input_size = num_classes = vocab_size = train_dataset.vocab_size#65
model = MambaLM(vocab_size).to(device)
print(sum(p.numel() for p in model.parameters() if p.requires_grad)) # 19683


137665


In [ ]:
# @title huggingface mamba2
# https://huggingface.co/docs/transformers/en/model_doc/mamba2
# https://github.com/huggingface/transformers/blob/main/src/transformers/models/mamba2/modeling_mamba2.py#L825
import torch
from transformers import Mamba2Config, Mamba2Model

# import torch.nn as nn
# class Id(nn.Module):
#     def __init__(self): super().__init__()
#     def forward(self, x): return x
# pred.embeddings = Id()

batch=4
seq_len=5
in_dim=132
d_model=128#256
# num_heads*head_dim=2*hidden_size ?

config = Mamba2Config(hidden_size=d_model, num_heads=8, head_dim=32, state_size=128, num_hidden_layers=1, output_hidden_states=True, vocab_size=0)
pred = Mamba2Model(config)
pred.embeddings = nn.Linear(in_dim, d_model)

x = torch.randn(batch, seq_len, in_dim)

# out = pred(inputs_embeds=x, use_cache=True)
out = pred(x, use_cache=True)
y, cache = out.last_hidden_state, out.cache_params
# print(x)
# print(y)
print(x.shape) # [batch, seq_len, d_model]
print(y.shape) # [batch, seq_len, d_model]
# print(len(h)) # num_hidden_layers + 1 ?
# h = torch.cat(out.hidden_states, dim=-1) # [batch, seq_len, (num_hidden_layers + 1 ?) * d_model]
# print(h.shape)
print(cache)

#  if key/value cache contains 10 tokens (no matter how many of it is a pad token), the cache position for the next token should be torch.tensor([10]) https://huggingface.co/docs/transformers/main/en/kv_cache
# out = pred(inputs_embeds=x, cache_params=cache, use_cache=True, cache_position=x.shape[1]) # torch.tensor([10])
out = pred(x, cache_params=cache, use_cache=True, cache_position=x.shape[1]) # torch.tensor([10])
y, cache = out.last_hidden_state, out.cache_params

print(y.shape)
# print(len(h)) # num_hidden_layers + 1 ?
print(cache.shape)




In [ ]:
# !pip install causal-conv1d mamba-ssm
# !pip install causal-conv1d==1.0.0
# !pip install mamba-ssm==1.0.1

!pip install torch==2.4.0 torchvision==0.19.0 torchaudio==2.4.0 --index-url https://download.pytorch.org/whl/cu121
!pip install mamba-ssm causal-conv1d
# !pip install causal-conv1d && pip install mamba-ssm

# !pip install causal-conv1d>=1.1.0 -q -U
# !pip install mamba-ssm -q -U


In [ ]:
# # import torch
# from mamba_ssm import Mamba2
# model = Mamba2(
#     d_model=128, # Model dimension d_model
#     d_state=64,  # SSM state expansion factor, typically 64 or 128
#     d_conv=4,    # Local convolution width
#     expand=2,    # Block expansion factor
# ).to("cuda")
# x=torch.randn(4, 100, 128).to("cuda")
# y = model(x)
# # assert y.shape == x.shape




# import torch
# from mamba_ssm import Mamba

batch, length, dim = 2, 100, 128
x = torch.randn(batch, length, dim).to("cuda")
# model = Mamba(
#     d_model=dim, # Model dimension d_model
#     d_state=16,  # SSM state expansion factor
#     d_conv=4,    # Local convolution width
#     expand=2,    # Block expansion factor
# ).to("cuda")
# y = model(x)



from mamba_ssm import Mamba2

model = Mamba2(
    # This module uses roughly 3 * expand * d_model^2 parameters
    d_model=dim, # Model dimension d_model
    d_state=64,  # SSM state expansion factor, typically 64 or 128
    d_conv=4,    # Local convolution width
    expand=2,    # Block expansion factor
    headdim=16, # just make sure d_model * expand / headdim = multiple of 8
).to("cuda")
y = model(x)





IndexError: map::at